In [64]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 35)
pd.set_option('display.width', 200)
pd.options.display.float_format = '{:,.2f}'.format

🐱‍💻 이번 챕터는`넘파이 배열`과 `판다스 배열`을 사용해 같은 작업을 처리하는 방법을 학습했음  
> '어떤 도구가 중요한지'는 개인적 취향보다는 **데이터의 성격**에 따라 결정된다

🐱‍💻 이번 챕터에서 다루는 Data
* **코로나19 일일 데이터** - 국가별 하루에 하나의 행 (신규 확진자, 신규 사망자) : 2019-12-31 ~ 2020-07-18
* **브라질 지표온도 데이터** - 브라질의 87개 기상관측소에서 매월 하나의 온도 판독값 : 2019-01-01 ~ 2019-12-31

# itertuples을 활용한 데이터 순회
* 이 방식대로 직접 구현하는 일이 거의 없더라도, 데이터의 행을 순회하고 그룹별로 정렬하는 개념을 알아두면 도움이 될 것임
> 참고) 10,000행 이하의 dataframe은 pandas 메서드로 처리하는 것이 순회(looping)보다 더 효율적임


## 코로나19 데이터

### 1. 데이터 로드

In [30]:
covidDaily = pd.read_csv('data/coviddaily720.csv', parse_dates=['casedate'])

In [31]:
covidDaily.sample(2)

,iso_code,casedate,location,continent,new_cases,new_deaths,population,pop_density,median_age,gdp_per_capita,hosp_beds,region
188,ALB,2020-03-12,Albania,Europe,1.00,1.00,"2,877,800.00",104.87,38.00,"11,803.43",2.89,Eastern Europe
8613,ERI,2020-06-23,Eritrea,Africa,0.00,0.00,"3,546,427.00",44.30,19.30,"1,510.46",0.70,East Africa


### 2. 열을 가독성이 좋게 정렬

In [16]:
covidDaily = covidDaily[['casedate', 'iso_code', 'location', 'region', 'continent', 'new_cases', 'new_deaths',\
                         'hosp_beds', 'population', 'pop_density', 'median_age', 'gdp_per_capita']]

### 3. 위치, 날짜별 정렬

In [72]:
covidDaily = covidDaily.sort_values(['location', 'casedate'])
covidDaily.head(3)

,iso_code,casedate,location,continent,new_cases,new_deaths,population,pop_density,median_age,gdp_per_capita,hosp_beds,region
0,AFG,2019-12-31,Afghanistan,Asia,0.00,0.00,"38,928,341.00",54.42,18.60,"1,803.99",0.50,South Asia
1,AFG,2020-01-01,Afghanistan,Asia,0.00,0.00,"38,928,341.00",54.42,18.60,"1,803.99",0.50,South Asia
2,AFG,2020-01-02,Afghanistan,Asia,0.00,0.00,"38,928,341.00",54.42,18.60,"1,803.99",0.50,South Asia


### 4. itertuples를 사용하여 행들을 순회  
* itertuple를 사용하여 전체 행과 이름없는 튜플들을 순회  
* 각국의 전체 날짜 대해 신규 확진자 수를 합산한다  
      
      e.g. 2019-12-31 ~ 2020-07-18 한국의 일일 신규 확진자 수 합계 

4-1. 🔍 **하단 코드 작동원리**
* 국가(location)가 바뀔 때마다 rowlist에 소계를 '딕셔너리' 형태로 추가한 뒤에, 카운트(casecnt)를 0으로 리셋한다  
* 예시
        첫번째 순회 시 casecnt = 0, prevloc = 'ZZZ', row.loction = 'Afghanistan', row.new_cases = 34451.0  
        두번째 순회 시 casecnt = 34451.0, prevloc = 'Afghanistan', row.location = 'Albania', row.new_cases = 3371.0

In [73]:
prevloc = 'ZZZ'
rowlist = []

for row in covidDaily.itertuples():
    # 조건 : 현재 순회 행(row)의 지역(location)이 이전 순회 행(row)과 달라지는 순간에
    if (prevloc!=row.location): 
        # 조건 : 초기값('ZZZ')은 예외로 두어 건너뛰도록 처리함
        if (prevloc!='ZZZ'): 
            # rowlist < '이전 지역'과 '이전 지역의 신규 확진자 수 누계'를 딕셔너리로 추가해둠
            rowlist.append({'location':prevloc, 'casecnt':casecnt}) 
        # 다음 지역에 대한 작업을 하기 위해서 케이스 빈도를 '0'으로 리셋
        casecnt = 0 
        # 다음 지역에 대한 작업을 하기 위해서, 지역을 '현재 순회 행(row)의 지역'으로 셋팅
        prevloc = row.location 
    # '현재 순회 행(row)의 신규 확진자 수'를 누적합 해준다 
    casecnt += row.new_cases 
    
# for문 가장 마지막 수행된 값은 저장되지 않기 때문에 > 반복문 빠져나온 뒤 저장되게 해줌    
rowlist.append({'location':prevloc, 'casecnt':casecnt})
len(rowlist)

209

In [55]:
rowlist[0:4]

[{'location': 'Afghanistan', 'casecnt': 34451.0},
 {'location': 'Albania', 'casecnt': 3371.0},
 {'location': 'Algeria', 'casecnt': 18712.0},
 {'location': 'Andorra', 'casecnt': 855.0}]

4-2. 📍  '딕셔너리'로 이루어진 리스트 > 데이터를 임시로 저장했다가 나중에 데이터프레임으로 변환하기에 좋다 

4-3. 💡 `itertuple`이란?  
- map 클래스의 메소드이다
- 데이터프레임 객체의 행을 튜플* 형식으로 읽어들이면서 순회한다
    * 리스트와 유사하게 여러 객체를 담고 있지만, 튜플 내의 값은 수정, 추가, 삭제가 안된다는 점이 차이점이다. ( )로 사용. 덮어씌우기는 가능하다. 
- 실제 출력값은 아래 코드결과 참고

In [41]:
number = 0 
for row in covidDaily.itertuples():
    number += 1
    if number > 2:
        continue
    print(number,". ", row)

1 .  Pandas(Index=0, iso_code='AFG', casedate=Timestamp('2019-12-31 00:00:00'), location='Afghanistan', continent='Asia', new_cases=0.0, new_deaths=0.0, population=38928341.0, pop_density=54.422, median_age=18.6, gdp_per_capita=1803.987, hosp_beds=0.5, region='South Asia')
2 .  Pandas(Index=1, iso_code='AFG', casedate=Timestamp('2020-01-01 00:00:00'), location='Afghanistan', continent='Asia', new_cases=0.0, new_deaths=0.0, population=38928341.0, pop_density=54.422, median_age=18.6, gdp_per_capita=1803.987, hosp_beds=0.5, region='South Asia')


### 5. rowlist(소계를 담고 있는 리스트) > dataframe

In [51]:
covidtotals = pd.DataFrame(rowlist)
covidtotals.head()

,location,casecnt
0,Afghanistan,"34,451.00"
1,Albania,"3,371.00"
2,Algeria,"18,712.00"
3,Andorra,855.00
4,Angola,483.00


## 브라질 지표온도 데이터 

### 1. 데이터 로드

In [89]:
ltBrazil = pd.read_csv('data/ltbrazil.csv')

### 2. 데이터 정렬 & 온도 누락값 행 제거
* 기상관측소(station)마다 월별(month)로 정렬되도록 한다

In [90]:
ltBrazil = ltBrazil.sort_values(['station','month'])
ltBrazil = ltBrazil.dropna(subset=['temperature'])

### 3. itertuples를 사용하여 행들을 튜플로 순회
* 🔍 **하단 코드 작동원리** : 평균 기온을 계산하고자 하는데, 그 해의 기온이 이전 달 기온보다 3도 넘게 높은/낮은 값은 제외한다.  

In [91]:
prevstation = 'ZZZ'
prevtemp = 0
rowlist = []

for row in ltBrazil.itertuples():
# 전체 기간 동안의 특정 관측소 최종 빈도(stationcnt), 평균 기온(avgtemp)을 rowlist에 저장
    #  조건 : 현재 순회 행(row)의 관측소가, 직전 순회 행(row)의 관측소와 달라진 순간에 저장함
    if (prevstation!=row.station):
        if (prevstation!='ZZZ'): # 초기값('ZZZ')일 때는 수행되지 않도록 함
            rowlist.append({'station':prevstation, 'avgtemp':tempcnt/stationcnt, 'stationcnt':stationcnt})
        tempcnt = 0
        stationcnt = 0
        prevstation = row.station
# 그 해까지 특정 관측소를 센 횟수와 더해진 기온값을 stationcnt, tempcnt에 저장  
    # [조건1 : 이전 온도와의 차이가 3도 이내인 행만 선택] + [조건2 : 관측소 최초 순회때는 무조건 수행되도록] 
    if ((0<=abs(row.temperature-prevtemp)<=3) or (stationcnt==0)):
        tempcnt += row.temperature
        stationcnt += 1
    prevtemp = row.temperature

# for문 맨 마지막 수행된 값도 저장 > 완료
rowlist.append({'station':prevstation, 'avgtemp':tempcnt/stationcnt, 'stationcnt':stationcnt})
rowlist[0:5]

[{'station': 'ALTAMIRA', 'avgtemp': 28.310000000000002, 'stationcnt': 5},
 {'station': 'ALTA_FLORESTA_AERO',
  'avgtemp': 29.433636363636367,
  'stationcnt': 11},
 {'station': 'ARAXA', 'avgtemp': 21.612499999999997, 'stationcnt': 4},
 {'station': 'BACABAL', 'avgtemp': 29.75, 'stationcnt': 4},
 {'station': 'BAGE', 'avgtemp': 20.366666666666664, 'stationcnt': 9}]

3-1. 중간과정

In [92]:
ltBrazil.head(2)

,locationid,year,month,temperature,latitude,longitude,elevation,station,countryid,country,latabs
648,BR000352000,2019,8,28.55,-3.20,-52.20,112.00,ALTAMIRA,BR,Brazil,3.20
740,BR000352000,2019,9,28.85,-3.20,-52.20,112.00,ALTAMIRA,BR,Brazil,3.20


In [93]:
num = 0

for row in ltBrazil.itertuples():
    num += 1
    if num>2:
        continue
    print(num,". ",row)
    

1 .  Pandas(Index=648, locationid='BR000352000', year=2019, month=8, temperature=28.55, latitude=-3.2, longitude=-52.2, elevation=112.0, station='ALTAMIRA', countryid='BR', country='Brazil', latabs=3.2)
2 .  Pandas(Index=740, locationid='BR000352000', year=2019, month=9, temperature=28.85, latitude=-3.2, longitude=-52.2, elevation=112.0, station='ALTAMIRA', countryid='BR', country='Brazil', latabs=3.2)


### 4. rowlist(평균 기온을 담고 있는 리스트) > dataframe

In [94]:
ltBrazilAvgs = pd.DataFrame(rowlist)
ltBrazilAvgs.head()

,station,avgtemp,stationcnt
0,ALTAMIRA,28.31,5
1,ALTA_FLORESTA_AERO,29.43,11
2,ARAXA,21.61,4
3,BACABAL,29.75,4
4,BAGE,20.37,9


### 5. itertuples를 사용하여 행들을 튜플로 순회  
* 🔍 하단 코드 작동원리 : 평균 기온을 계산 (+*`월별 기온 변화`*의 폭이 어떻든지에 상관없이)

In [95]:
prevstation = 'ZZZ'
rowlist_s = []

for row in ltBrazil.itertuples():
# 전체 기간 동안의 특정 관측소 최종 빈도(stationcnt), 평균 기온(avgtemp)을 rowlist에 저장
    #  조건 : 현재 순회 행(row)의 관측소가, 직전 순회 행(row)의 관측소와 달라진 순간에 저장함
    if (prevstation!=row.station):
        if (prevstation!='ZZZ'): # 초기값('ZZZ')일 때는 수행되지 않도록 함
            rowlist_s.append({'station':prevstation, 'avgtemp':tempcnt/stationcnt, 'stationcnt':stationcnt})
        tempcnt = 0
        stationcnt = 0
        prevstation = row.station
    tempcnt += row.temperature
    stationcnt += 1

# for문 맨 마지막 수행된 값도 저장 > 완료
rowlist_s.append({'station':prevstation, 'avgtemp':tempcnt/stationcnt, 'stationcnt':stationcnt})
rowlist_s[0:5]

[{'station': 'ALTAMIRA', 'avgtemp': 28.310000000000002, 'stationcnt': 5},
 {'station': 'ALTA_FLORESTA_AERO',
  'avgtemp': 29.37416666666667,
  'stationcnt': 12},
 {'station': 'ARAXA', 'avgtemp': 21.612499999999997, 'stationcnt': 4},
 {'station': 'BACABAL', 'avgtemp': 29.75, 'stationcnt': 4},
 {'station': 'BAGE', 'avgtemp': 19.29583333333333, 'stationcnt': 12}]

### 6. rowlist_s(모든 기록의 평균 기온을 담고 있는 리스트) > dataframe

In [96]:
ltBrazilAvgsShallow = pd.DataFrame(rowlist_s)
ltBrazilAvgsShallow.head()

,station,avgtemp,stationcnt
0,ALTAMIRA,28.31,5
1,ALTA_FLORESTA_AERO,29.37,12
2,ARAXA,21.61,4
3,BACABAL,29.75,4
4,BAGE,19.30,12


### 7. 잘 작동되었는지 비교 확인 (3번 결과 - 4번 결과)

In [97]:
ltBrazilAvgs[ltBrazilAvgs.station=='BAGE']

,station,avgtemp,stationcnt
4,BAGE,20.37,9


In [98]:
ltBrazilAvgsShallow[ltBrazilAvgsShallow.station=='BAGE']

,station,avgtemp,stationcnt
4,BAGE,19.30,12


* BAGE 관측소에 3도 넘게 기온변화가 있는 데이터는 총 3개가 있음 (index = 485, 642, 918)

In [99]:
ltBrazil[ltBrazil.station=='BAGE']

,locationid,year,month,temperature,latitude,longitude,elevation,station,countryid,country,latabs
90,BRM00083980,2019,1,25.10,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33
182,BRM00083980,2019,2,23.90,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33
274,BRM00083980,2019,3,21.80,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33
366,BRM00083980,2019,4,20.15,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33
458,BRM00083980,2019,5,16.90,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33
550,BRM00083980,2019,6,17.45,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33
642,BRM00083980,2019,7,12.40,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33
734,BRM00083980,2019,8,13.50,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33
826,BRM00083980,2019,9,15.75,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33
918,BRM00083980,2019,10,18.95,-31.33,-54.10,242.00,BAGE,BR,Brazil,31.33


# numpy array로 그룹별 합계 계산
* 앞에서 itertuple로 했던 작업을 넘파이 배열을 사용하여 수행할 수도 있다.

## 코로나19 데이터

### 데이터 로드

In [137]:
covidDaily = pd.read_csv("data/coviddaily720.csv", parse_dates=["casedate"])

In [138]:
covidDaily.head(3)

,iso_code,casedate,location,continent,new_cases,new_deaths,population,pop_density,median_age,gdp_per_capita,hosp_beds,region
0,AFG,2019-12-31,Afghanistan,Asia,0.00,0.00,"38,928,341.00",54.42,18.60,"1,803.99",0.50,South Asia
1,AFG,2020-01-01,Afghanistan,Asia,0.00,0.00,"38,928,341.00",54.42,18.60,"1,803.99",0.50,South Asia
2,AFG,2020-01-02,Afghanistan,Asia,0.00,0.00,"38,928,341.00",54.42,18.60,"1,803.99",0.50,South Asia


### 1. 위치 리스트 생성하기

In [139]:
loclist = covidDaily.location.unique().tolist()

### 2. 위치별 합계를 계산하기

2-0. 위치 리스트 확인

In [135]:
loclist[0:5]

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola']

2-1. 위치별 일일 신규 확진자 수를 모두 cases 리스트에 담고 > sum(cases)로 한꺼번에 합계 연산 ⭐

In [133]:
rowlist = []
casevalues = covidDaily[['location','new_cases']].to_numpy()

for locitem in loclist:
    # 위치별(casevalues[j][0])로 모든 신규 확진자 수 값(casevalues[j][1])을 선택
    cases = [casevalues[j][1] for j in range(len(casevalues)) if casevalues[j][0]==locitem]
    # cases에 담긴 모든 값을 더한 뒤 > rowlist의 값으로 추가해둠
    rowlist.append(sum(cases))
    
len(rowlist), len(loclist)

(209, 209)

* 2-1의 중간 과정

In [108]:
covidDaily[['location','new_cases']].to_numpy()

array([['Afghanistan', 0.0],
       ['Afghanistan', 0.0],
       ['Afghanistan', 0.0],
       ...,
       ['Zimbabwe', 41.0],
       ['Zimbabwe', 16.0],
       ['Zimbabwe', 40.0]], dtype=object)

In [128]:
casevalues[0][0], casevalues[184][0], casevalues[185][0]

('Afghanistan', 'Afghanistan', 'Albania')

In [114]:
len(covidDaily), len(casevalues)

(29213, 29213)

2-2. 합계값 확인

In [134]:
rowlist[0:5]

[34451.0, 3371.0, 18712.0, 855.0, 483.0]

### 3. lists > zip 내장함수로 묶기 > dataframe

In [112]:
casetotals = pd.DataFrame(zip(loclist,rowlist), columns=(['location','casetotals']))
casetotals.head()

,location,casetotals
0,Afghanistan,982.00


# groupby로 데이터를 그룹별 조직화
* 앞의 방법에서 다룬 방법으로도 요약통계를 생성할 수 있으나
* pandas dataframe의 **📍groupby 메서드**를 활용하여 집계 작업하는 것이 일반적이고 효율도 높다.

## 코로나19 데이터

In [136]:
covidDaily = pd.read_csv("data/coviddaily720.csv", parse_dates=["casedate"])

### 1. 판다스 groupby `데이터 프레임`을 생성하기

In [140]:
countryTots = covidDaily.groupby(['location'])
type(countryTots)

pandas.core.groupby.generic.DataFrameGroupBy

In [144]:
countryTots

### 2. 국가별로 첫 행과 마지막 행의 `데이터 프레임` 생성하기

In [146]:
countryTots.first().iloc[0:5, 0:5]

,iso_code,casedate,continent,new_cases,new_deaths
location,,,,,
Afghanistan,AFG,2019-12-31,Asia,0.00,0.00
Albania,ALB,2020-03-09,Europe,2.00,0.00
Algeria,DZA,2019-12-31,Africa,0.00,0.00
Andorra,AND,2020-03-03,Europe,1.00,0.00
Angola,AGO,2020-03-22,Africa,2.00,0.00


In [147]:
countryTots.last().iloc[0:5, 0:5]

,iso_code,casedate,continent,new_cases,new_deaths
location,,,,,
Afghanistan,AFG,2020-07-12,Asia,85.00,16.00
Albania,ALB,2020-07-12,Europe,93.00,4.00
Algeria,DZA,2020-07-12,Africa,904.00,16.00
Andorra,AND,2020-07-12,Europe,0.00,0.00
Angola,AGO,2020-07-12,Africa,25.00,2.00


In [148]:
type(countryTots.last())

pandas.core.frame.DataFrame

### 3. 국가별로 전체 행 얻기

In [150]:
countryTots.get_group('Zimbabwe').iloc[0:5, 0:5]

,iso_code,casedate,continent,new_cases,new_deaths
29099,ZWE,2020-03-21,Africa,1.00,0.00
29100,ZWE,2020-03-22,Africa,1.00,0.00
29101,ZWE,2020-03-23,Africa,0.00,0.00
29102,ZWE,2020-03-24,Africa,0.00,1.00
29103,ZWE,2020-03-25,Africa,0.00,0.00


### 4. 그룹에 대해 순회(looping)

In [152]:
for name, group in countryTots:
  if (name in ['Malta','Kuwait']):
    print(group.iloc[0:5, 0:5])

      iso_code   casedate location continent  new_cases
14707      KWT 2019-12-31   Kuwait      Asia       0.00
14708      KWT 2020-01-01   Kuwait      Asia       0.00
14709      KWT 2020-01-02   Kuwait      Asia       0.00
14710      KWT 2020-01-03   Kuwait      Asia       0.00
14711      KWT 2020-01-04   Kuwait      Asia       0.00
      iso_code   casedate location continent  new_cases
17057      MLT 2020-03-07    Malta    Europe       1.00
17058      MLT 2020-03-08    Malta    Europe       2.00
17059      MLT 2020-03-09    Malta    Europe       0.00
17060      MLT 2020-03-10    Malta    Europe       2.00
17061      MLT 2020-03-11    Malta    Europe       1.00


### 5. 국가별 행 수 보기

In [154]:
countryTots.size()

location
Afghanistan       185
Albania           126
Algeria           190
Andorra           121
Angola            113
                 ... 
Vietnam           191
Western Sahara     78
Yemen              94
Zambia            116
Zimbabwe          114
Length: 209, dtype: int64

### 6. 국가별 요약통게 보기

6-1. 기본 요약통계

In [155]:
countryTots.new_cases.describe().head()

,count,mean,std,min,25%,50%,75%,max
location,,,,,,,,
Afghanistan,185.00,186.22,257.31,0.00,0.00,37.00,302.00,"1,063.00"
Albania,126.00,26.75,24.65,0.00,9.00,17.00,35.50,93.00
Algeria,190.00,98.48,123.98,0.00,0.00,88.00,149.75,904.00
Andorra,121.00,7.07,12.72,0.00,0.00,1.00,9.00,79.00
Angola,113.00,4.27,8.51,0.00,0.00,1.00,5.00,62.00


6-2. 합계

In [156]:
countryTots.new_cases.sum().head()

location
Afghanistan   34,451.00
Albania        3,371.00
Algeria       18,712.00
Andorra          855.00
Angola           483.00
Name: new_cases, dtype: float64